In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os, pandas as pd

BUG_CSV = "/content/drive/MyDrive/java_bug_detection.csv"

assert os.path.exists(BUG_CSV), f"CSV not found at {BUG_CSV}"
print("Found:", BUG_CSV)

Found: /content/drive/MyDrive/java_bug_detection.csv


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv(BUG_CSV)

# Keep only bug_detection rows if your file contains multiple tasks
if "task" in df.columns:
    df = df[df["task"] == "bug_detection"].copy()

# Basic cleaning
df = df.dropna(subset=["input","output"])
df = df[(df["input"].str.strip()!="") & (df["output"].str.strip()!="")].reset_index(drop=True)

# Optional: limit max rows for a first run (uncomment if you want a quick test)
# df = df.sample(n=min(20000, len(df)), random_state=42).reset_index(drop=True)

train_df, tmp_df = train_test_split(df, test_size=0.2, random_state=42)
val_df, test_df   = train_test_split(tmp_df, test_size=0.5, random_state=42)
len(train_df), len(val_df), len(test_df)


(6372, 796, 797)

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

MODEL_NAME = "Salesforce/codet5p-220m"  # smaller CodeT5+; good for Colab
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

# Memory-friendly tweaks
model.gradient_checkpointing_enable()
tokenizer.padding_side = "right"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

print("Device:", device)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/768 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/446M [00:00<?, ?B/s]

Device: cuda


In [ ]:
from torch.utils.data import Dataset

MAX_SOURCE_LEN = 512   # adjust up/down if you OOM / want more context
MAX_TARGET_LEN = 512

class BugFixDataset(Dataset):
    def __init__(self, frame):
        self.inputs = frame["input"].tolist()
        self.labels = frame["output"].tolist()

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        src = str(self.inputs[idx])
        tgt = str(self.labels[idx])

        enc = tokenizer(
            src,
            max_length=MAX_SOURCE_LEN,
            truncation=True,
            padding=False,
            return_tensors=None,
        )
        with tokenizer.as_target_tokenizer():
            dec = tokenizer(
                tgt,
                max_length=MAX_TARGET_LEN,
                truncation=True,
                padding=False,
                return_tensors=None,
            )
        return {
            "input_ids": enc["input_ids"],
            "attention_mask": enc["attention_mask"],
            "labels": dec["input_ids"],
        }

train_ds = BugFixDataset(train_df)
val_ds   = BugFixDataset(val_df)
test_ds  = BugFixDataset(test_df)
len(train_ds), len(val_ds), len(test_ds)


(6372, 796, 797)

In [ ]:
from torch.nn.utils.rnn import pad_sequence

PAD = tokenizer.pad_token_id

def collate_fn(batch):
    input_ids = [torch.tensor(x["input_ids"], dtype=torch.long) for x in batch]
    attn_mask = [torch.tensor(x["attention_mask"], dtype=torch.long) for x in batch]
    labels    = [torch.tensor(x["labels"], dtype=torch.long) for x in batch]

    input_ids = pad_sequence(input_ids, batch_first=True, padding_value=PAD)
    attn_mask = pad_sequence(attn_mask, batch_first=True, padding_value=0)
    labels    = pad_sequence(labels, batch_first=True, padding_value=PAD)

    # Replace label pad with -100 (ignored by loss)
    labels = labels.masked_fill(labels == PAD, -100)
    return {
      "input_ids": input_ids,
      "attention_mask": attn_mask,
      "labels": labels
    }


In [ ]:
from torch.utils.data import DataLoader

BATCH_SIZE = 2  # keep small for Colab; use grad_accum to raise effective batch
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn, num_workers=2)
val_loader   = DataLoader(val_ds,   batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn, num_workers=2)


In [ ]:
from transformers import get_linear_schedule_with_warmup

EPOCHS = 3
LR = 2e-5
WEIGHT_DECAY = 0.01
GRAD_ACCUM_STEPS = 8       # effective batch = BATCH_SIZE * GRAD_ACCUM_STEPS
MAX_GRAD_NORM = 1.0
WARMUP_RATIO = 0.1

optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)

num_update_steps_per_epoch = (len(train_loader) + GRAD_ACCUM_STEPS - 1) // GRAD_ACCUM_STEPS
t_total = EPOCHS * num_update_steps_per_epoch
num_warmup = int(WARMUP_RATIO * t_total)

scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup, t_total)

scaler = torch.cuda.amp.GradScaler(enabled=torch.cuda.is_available())  # mixed precision
torch.backends.cuda.matmul.allow_tf32 = True


/tmp/ipython-input-3050780555.py:18: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=torch.cuda.is_available())  # mixed precision


In [ ]:
pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 7.4 MB/s eta 0:00:00


In [ ]:
import math, time, torch.nn as nn

def evaluate(model, loader, max_gen_tokens=128, max_batches=50):
    model.eval()
    preds, refs = [], []
    start_time = time.time()
    with torch.no_grad():
        for i, batch in enumerate(loader):
            if i >= max_batches:   # safety stop
                break
            # ⏱ timeout safeguard (stop if >5 min in val)
            if time.time() - start_time > 300:
                print("⚠️ Validation stopped early (timeout safeguard)")
                break

            batch = {k:v.to(device) for k,v in batch.items()}
            gen = model.generate(
                input_ids=batch["input_ids"],
                attention_mask=batch["attention_mask"],
                max_new_tokens=max_gen_tokens,
                num_beams=1,   # greedy decoding (fast)
            )
            pred_text = tokenizer.batch_decode(gen, skip_special_tokens=True)
            labels = batch["labels"].clone()
            labels[labels == -100] = PAD
            ref_text = tokenizer.batch_decode(labels, skip_special_tokens=True)
            preds.extend([p.strip() for p in pred_text])
            refs.extend([r.strip() for r in ref_text])
    return preds, refs

def train():
    global_step = 0
    model.train()
    ce = nn.CrossEntropyLoss(ignore_index=-100)

    for epoch in range(EPOCHS):
        print(f"\nEpoch {epoch+1}/{EPOCHS}")
        epoch_loss = 0.0
        optimizer.zero_grad(set_to_none=True)

        for step, batch in enumerate(train_loader, start=1):
            batch = {k:v.to(device) for k,v in batch.items()}

            with torch.amp.autocast("cuda", enabled=torch.cuda.is_available()):
                outputs = model(**batch)
                loss = outputs.loss / GRAD_ACCUM_STEPS

            scaler.scale(loss).backward()
            epoch_loss += loss.item() * GRAD_ACCUM_STEPS

            if step % GRAD_ACCUM_STEPS == 0:
                scaler.unscale_(optimizer)
                torch.nn.utils.clip_grad_norm_(model.parameters(), MAX_GRAD_NORM)
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad(set_to_none=True)
                scheduler.step()
                global_step += 1

            if step % (GRAD_ACCUM_STEPS * 20) == 0:
                print(f"  step {step:>6} | loss {epoch_loss/step:.4f}")

        print(f"Epoch avg loss: {epoch_loss/len(train_loader):.4f}")

        # Fast validation
        preds, refs = evaluate(model, val_loader, max_gen_tokens=128, max_batches=50)
        from sacrebleu import corpus_bleu
        bleu = corpus_bleu(preds, [refs]).score
        exact = sum(1 for p, r in zip(preds, refs) if p == r) / max(1, len(refs)) * 100.0
        print(f"Validation — BLEU: {bleu:.2f} | Exact match: {exact:.2f}%")

train()



Epoch 1/3


/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4006: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4006: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


  step    160 | loss 0.0623
  step    320 | loss 0.0589
  step    480 | loss 0.0563
  step    640 | loss 0.0561
  step    800 | loss 0.0562
  step    960 | loss 0.0556
  step   1120 | loss 0.0563
  step   1280 | loss 0.0557
  step   1440 | loss 0.0551
  step   1600 | loss 0.0552
  step   1760 | loss 0.0541
  step   1920 | loss 0.0538
  step   2080 | loss 0.0530
  step   2240 | loss 0.0528
  step   2400 | loss 0.0527
  step   2560 | loss 0.0523
  step   2720 | loss 0.0528
  step   2880 | loss 0.0531
  step   3040 | loss 0.0526
Epoch avg loss: 0.0524


/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4006: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4006: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Validation — BLEU: 6.94 | Exact match: 0.00%

Epoch 2/3


...

Validation — BLEU: 6.94 | Exact match: 0.00%

Epoch 3/3


/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4006: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4006: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


  step    160 | loss 0.0342
  step    320 | loss 0.0307
  step    480 | loss 0.0292
  step    640 | loss 0.0285
  step    800 | loss 0.0276
  step    960 | loss 0.0279
  step   1120 | loss 0.0280
  step   1280 | loss 0.0285
  step   1440 | loss 0.0282
  step   1600 | loss 0.0282
  step   1760 | loss 0.0284
  step   1920 | loss 0.0285
  step   2080 | loss 0.0279
  step   2240 | loss 0.0279
  step   2400 | loss 0.0281
  step   2560 | loss 0.0281
  step   2720 | loss 0.0283
  step   2880 | loss 0.0286
  step   3040 | loss 0.0286
Epoch avg loss: 0.0288


/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4006: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4006: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Validation — BLEU: 6.94 | Exact match: 0.00%


In [ ]:
OUT_DIR = "/content/drive/MyDrive/debugging_runs"
os.makedirs(OUT_DIR, exist_ok=True)

In [ ]:
# Save predictions
pred_csv = os.path.join(OUT_DIR, "test_predictions.csv")
out_df.to_csv(pred_csv, index=False, encoding="utf-8")
print("✅ Saved predictions to:", pred_csv)

# Save model + tokenizer
save_dir = os.path.join(OUT_DIR, "codet5p_bugfix_finetuned")
model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)
print("✅ Saved model to:", save_dir)


✅ Saved predictions to: /content/drive/MyDrive/debugging_runs/test_predictions.csv
✅ Saved model to: /content/drive/MyDrive/debugging_runs/codet5p_bugfix_finetuned


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Path where you saved earlier
MODEL_PATH = "/content/drive/MyDrive/debugging_runs/codet5p_bugfix_finetuned"

# Load from Drive
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, use_fast=True)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_PATH)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

print("✅ Model loaded from:", MODEL_PATH)


✅ Model loaded from: /content/drive/MyDrive/debugging_runs/codet5p_bugfix_finetuned


In [ ]:
def predict(text, max_tokens=128):
    inputs = tokenizer(text, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_tokens,
            num_beams=1,        # greedy decoding
            do_sample=False,    # deterministic
            num_return_sequences=1
        )
    pred = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # 🧹 Clean extra completions → keep only the first valid Java class/method
    if "{" in pred and "}" in pred:
        start = pred.find("public")   # first occurrence of a class/method
        end = pred.rfind("}") + 1     # last closing brace
        if start != -1 and end != -1:
            pred = pred[start:end]

    return pred.strip()


# Example 1: Buggy Java code (logical error: subtract instead of add)
buggy_java = """
public class Calculator {
    public static int sub(int a,int b) {
        return a+b;
    }
}
"""

print("🔴 Buggy Input (Java):")
print(buggy_java)

prediction = predict(buggy_java)
print("\n✅ Model Prediction (fixed Java code):")
print(prediction)


🔴 Buggy Input (Java):

public class Calculator {
    public static int sub(int a,int b) {
        return a+b;
    }
}


✅ Model Prediction (fixed Java code):
public class Calculator {
    public static int sub(int a,int b) {
        return a-b;
    }
}

public class SubtractingInt {
    public static int sub(int a,int b) {
        return a-b;
    }
}

public class Calculator2 {
    public static int sub(int a,int b) {
        return a+b;
    }
}


In [ ]:
# 1. Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# 2. Load CSV
import pandas as pd

# Change this path to your file location inside Drive
csv_path = "/content/drive/MyDrive/java_bug_detection.csv"

df = pd.read_csv(csv_path)

# 3. Check dataset info
print("Columns:", df.columns.tolist())
print("Number of rows:", len(df))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Columns: ['task', 'bug_type', 'repo', 'input', 'output']
Number of rows: 7965
